# Setup Instructions

This notebook requires few things before you start:
1. **Dataset**
2. **Compiled test code** (`special_functions.cpython-312.pyc`)
3. **The python version should be** `3.12.x`

They will be automatically downloaded in the next cell.

If the download fails for any reason:
- You can download the files manually to your computer with the link below.  
- Then upload them into Colab using the **Upload to Session Storage**.  
- Make sure that:
  - The dataset is placed in a folder called `data/`
  - The compiled test code is placed inside a folder called `__pycache__/`


In [17]:
# Download test code
!wget -O __pycache__.zip "https://kth-my.sharepoint.com/:u:/g/personal/haotianq_ug_kth_se/EQ5EmlG2qP5DhRr9e9PKYZ8BWuhCPA34FIifFKCUK_BK_A?e=Gp1jXc&download=1"
!unzip __pycache__.zip

# Download dataset
!wget -O data.zip "https://kth-my.sharepoint.com/:u:/g/personal/haotianq_ug_kth_se/EYHf_bkJRN5CgrMtKacEr7sBlSDK63WvgQ76xB4N_FWqkg?e=ReixTF&download=1"

zsh:1: command not found: wget
Archive:  __pycache__.zip
   creating: __pycache__
  inflating: __pycache__/special_functions.cpython-312.pyc  
zsh:1: command not found: wget


In [3]:
import importlib.util, sys

if sys.version_info.major == 3 and sys.version_info.minor == 12:
    print("Python version is OK:", sys.version.split()[0])
else:
    print("Python version is NOT 3.12.x, found:", sys.version.split()[0])

spec = importlib.util.spec_from_file_location(
    "special_functions",
    "__pycache__/special_functions.cpython-312.pyc"
)
module = importlib.util.module_from_spec(spec)
sys.modules["special_functions"] = module
spec.loader.exec_module(module)

Python version is OK: 3.12.11


## Name and birthday

**Fill in your name, birthday, and *run* the code below.** The birthday should be of the format YYYY-MM-DD. It is important that it is correct, as we will use it to fix the random seed later.

In [4]:
student_name = "Rios_Omar"
student_birthday = "2000-05-24"

# Assignment 1: Synthesis problems and synthesis principles

In this assignment, you will complete a few tasks that will introduce you to some of the basic synthesis problems and synthesis principles that we have discussed in the lecture. Hopefully, we will see some of the concepts from lecture 1 in practice. Begin by importing the required packages.

In [ ]:
from special_functions import * # <- TODO MAY NEED TO CHANGE NAME DEPENDING ON FILE NAME
import os
import csv
import torch
import torchvision
import random
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.mixture import GaussianMixture

birthday_int = int(student_birthday.replace("-", ""))

In [ ]:
import zipfile

zip_path = "data.zip"

if os.path.exists(zip_path):
    extract_to = os.path.splitext(zip_path)[0]
    os.makedirs(extract_to, exist_ok=True)

    # Extract all files
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

    print(f"Files extracted to {extract_to}")
else:
    pass

## Load Data

Now, let us load and visualise our data. In this exercise, we will be using the CelebA dataset, which contains 202,599 images of celebrities. The dataset is already partitioned into training, validation (evaluation), and test sets. Moreover, the CelebA dataset includes attribute labels for each image, making it easier to filter the images if we want or have the need to. For example, some of the attributes indicate whether the celebrity is wearing glasses, has a beard, is smiling, or is bald.

We will load the data using the PyTorch DataLoader approach. Doing it this way not only allows us to shuffle the data efficiently, but also enables us to load the data in batches. Batch loading will be crucial for later parts of the course, as it allows for parallel processing on GPUs, speeding up computation when we deal with more complicated models later on. Therefore, while this approach is not strictly necessary in this exercise (as we will see), it is good to be familiar with it already.

First, we will create the class that will handle loading and preprocessing each image in the dataset. Then, we will wrap it with a PyTorch DataLoader. The first cell below is the class. The second cell instantiate our dataset into two sets: a validation and a training set, before wrapping them around dataloaders.

In [ ]:
class CelebADataset(Dataset):
    def __init__(self, data_dir, attr_file=None, bbox_file=None, eval_file=None, partition=None,
                 transform=None, filter_attrs=None, keep_percentage=1.0):
        """
        Custom dataset for CelebA with optional filtering.

        Parameters:
        - data_dir (str): Path to the image directory.
        - attr_file (str or None): Path to the attributes CSV. If None, no attribute filtering is applied.
        - bbox_file (str or None): Path to the bounding box CSV. If None, no cropping is applied.
        - eval_file (str or None): Path to the evaluation partition CSV. If None, all images are used.
        - partition (int or None): Dataset partition to use (0=train, 1=eval, 2=test). If None, all partitions are used.
        - transform (torchvision.transforms): Image transformations.
        - filter_attrs (dict or None): Attributes to filter. Example: {'Smiling': 1, 'Eyeglasses': 1}
        - keep_percentage (float): Fraction of selected images to keep (0.0 to 1.0).
        """
        self.data_dir = data_dir
        self.transform = transform

        # Load partition data if specified
        all_images = set(os.listdir(data_dir))
        if eval_file and partition is not None:
            partition_data = self._load_partition_data(eval_file)
            all_images = {img for img, part in partition_data.items() if part == partition}

        # Load bounding boxes if provided
        self.bbox_data = self._load_bbox_data(bbox_file) if bbox_file else {}

        # Load and filter attributes if provided
        if attr_file and filter_attrs:
            attr_data = self._load_attr_data(attr_file)
            self.image_files = [
                img for img in all_images if all(attr_data.get(img, {}).get(attr, 0) == value for attr, value in filter_attrs.items())
            ]
        else:
            self.image_files = list(all_images)  # Use all images if no filtering

        # Apply percentage filtering
        total_images = len(self.image_files)
        keep_size = int(total_images * keep_percentage)
        self.image_files = self.image_files[:keep_size]

    def _load_partition_data(self, eval_file):
        """Loads partition data from the evaluation CSV."""
        partition_data = {}
        with open(eval_file, 'r') as f:
            reader = csv.reader(f)
            next(reader)  # Skip header

            for row in reader:
                img_name, part = row[0], int(row[1])
                partition_data[img_name] = part

        return partition_data

    def _load_bbox_data(self, bbox_file):
        """Loads bounding box data from a CSV file."""
        bbox_data = {}
        with open(bbox_file, 'r') as f:
            reader = csv.reader(f)
            next(reader)  # Skip header

            for row in reader:
                img_name, x, y, w, h = row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4])
                bbox_data[img_name] = {'x': x, 'y': y, 'w': w, 'h': h}

        return bbox_data

    def _load_attr_data(self, attr_file):
        """Loads attribute data from a CSV file."""
        attr_data = {}
        with open(attr_file, 'r') as f:
            reader = csv.reader(f)
            header = next(reader)[1:]  # Get attribute names (skip 'image_id')

            for row in reader:
                img_name = row[0]
                attr_values = [int(x) for x in row[1:]]  # Convert attributes to int (-1 or 1)
                attr_data[img_name] = {header[i]: attr_values[i] for i in range(len(header))}

        return attr_data

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.data_dir, img_name)

        # Load image
        image = Image.open(img_path).convert("RGB")

        # Apply bounding box cropping if bbox data is available
        if img_name in self.bbox_data:
            bbox = self.bbox_data[img_name]
            x, y, w, h = bbox['x'], bbox['y'], bbox['w'], bbox['h']

            # Ensure bounding box coordinates are within image size
            image = image.crop((x, y, x + w, y + h))

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        return image

In [ ]:
pixel_dim = 64

# Define transformations
transform = transforms.Compose([
    transforms.CenterCrop(140),
    transforms.Resize((pixel_dim, pixel_dim)),  #
    transforms.ToTensor(),
])

# Create train dataset with the images:
dataset = CelebADataset(
    data_dir='data/img_align_celeba/img_align_celeba',
    attr_file='data/list_attr_celeba.csv',
    bbox_file=None,  # Set to None to disable cropping
    eval_file='data/list_eval_partition.csv',
    partition=0,  # Use train set (0=train, 1=eval, 2=test)
    transform=transform,
    filter_attrs={},
    keep_percentage=0.02 # TODO -> CHANGE THIS! (0.002 = 325 images)
)

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Create evaluation dataset with the images:
dataset_eval = CelebADataset(
    data_dir='data/img_align_celeba/img_align_celeba',
    attr_file='data/list_attr_celeba.csv',
    bbox_file=None,  # Set to None to disable cropping
    eval_file='data/list_eval_partition.csv',
    partition=1,  # Use train set (0=train, 1=eval, 2=test)
    transform=transform,
    filter_attrs={},
    keep_percentage=0.1 # TODO -> CHANGE THIS!
)

# Create a DataLoader for the evaluation images
dataloader_eval = DataLoader(dataset_eval, batch_size=64, shuffle=True)

The batchsize of each dataloader is 64, meaning that each batch contains 64 images. Notice how we have transformed each image into 64x64 pixels by first centring the image and then cropping them (see the transform we have defined). Each image therefore contains 64x64x3 pixel values: 64x64 pixels with 3 RGB channels.

Now that we have initialised the dataset and created the dataloader, let's display some of our images.

In [ ]:
# Verify the dataset
# Get a batch of images
data_iter = iter(dataloader)
images = next(data_iter)

# Display the images
grid_img = torchvision.utils.make_grid(images, nrow=8)
plt.figure(figsize=(15, 15))
plt.imshow(grid_img.permute(1, 2, 0))
plt.axis('off')
plt.show()

In our current implementation, each image is a PyTorch tensor with shape (3, 64, 64). For this exercise, however, we will mostly be using the Scikit-Learn library. Many of the built-in functions in Scikit-Learn requires the data to be in numpy arrays rather than PyTorch tensors. The cell below transforms our train and validation datasets into numpy arrays. The variables `dataset_np` and `evalset_np`, which are the two datasets we will primarily work on, are the resulting data in numpy arrays. Make sure you understand what is happening, as it will greatly aid understanding for later tasks.

In [ ]:
dataset_np = []
evalset_np = []

for batch in dataloader:
    # batch has shape [batch_size, 3, 64, 64]
    dataset_np.append(batch)

for batch in dataloader_eval:
    evalset_np.append(batch)

# Make it so everything is fit along sample dimension
# shape: [n_images, 3, 64, 64]
dataset_np = torch.cat(dataset_np, dim=0)
evalset_np = torch.cat(evalset_np, dim=0)

# Transforms the pyTorch tensors the numpy arrays.
dataset_np = dataset_np.numpy()
evalset_np = evalset_np.numpy()

# Transpose to RGB-last for image flattening
# shape: [n_images, 64, 64, 3]
dataset_np = dataset_np.transpose(0, 2, 3, 1)
evalset_np = evalset_np.transpose(0, 2, 3, 1)

# Final flattening
# shape: (n_images, 12288)
dataset_np = dataset_np.reshape(dataset_np.shape[0], -1)
evalset_np = evalset_np.reshape(evalset_np.shape[0], -1)

Now that we have transformed our data into the required shape and format, we are ready to begin! Good luck!

## 1.1 Mass covering and mode collapse

We’ll begin this exercise with a simple game. The first cell implements a Gaussian Mixture Model (GMM) with variance flooring (try to think about why this may be useful?). Although scikit-learn offers a GMM implementation, it doesn’t include variance flooring by default.

Below that, the second cell implements three different models for generating face images.<br/>
Model A: samples directly from the original training dataset.<br/>
Model B: samples from a GMM fitted to the entire training dataset.<br/>
Model C: samples from a GMM fitted to only a few faces and with a low number of GMM-components.<br/>

In [ ]:
from sklearn.mixture._gaussian_mixture import _estimate_gaussian_parameters
from sklearn.mixture._gaussian_mixture import _compute_precision_cholesky

class GaussianMixtureWithVarianceFloor(GaussianMixture):
    """
    GaussianMixture subclass that applies variance flooring during the M-step.
    Only supports diagonal covariance type.
    """
    def __init__(
        self,
        n_components=1,
        covariance_type="diag",
        tol=1e-3,
        reg_covar=1e-6,
        max_iter=100,
        n_init=1,
        init_params="kmeans",
        weights_init=None,
        means_init=None,
        precisions_init=None,
        random_state=None,
        warm_start=False,
        verbose=0,
        verbose_interval=10,
        variance_floor=1e-3
    ):
        super().__init__(
            n_components=n_components,
            covariance_type=covariance_type,
            tol=tol,
            reg_covar=reg_covar,
            max_iter=max_iter,
            n_init=n_init,
            init_params=init_params,
            weights_init=weights_init,
            means_init=means_init,
            precisions_init=precisions_init,
            random_state=random_state,
            warm_start=warm_start,
            verbose=verbose,
            verbose_interval=verbose_interval
        )
        self.variance_floor = variance_floor

    def _m_step(self, X, log_resp):
        """M step.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)

        log_resp : array-like of shape (n_samples, n_components)
            Logarithm of the posterior probabilities (or responsibilities) of
            the point of each sample in X.
        """
        self.weights_, self.means_, self.covariances_ = _estimate_gaussian_parameters(
            X, np.exp(log_resp), self.reg_covar, self.covariance_type
        )
        self.weights_ /= self.weights_.sum()

        if self.covariance_type == "diag":
          self.covariances_ = np.maximum(self.covariances_, self.variance_floor)
        else:
           raise ValueError("GaussianMixtureWithVarianceFloor only supports 'diag' covariance_type.")

        self.precisions_cholesky_ = _compute_precision_cholesky(
            self.covariances_, self.covariance_type)

In [ ]:
def model_A(dataset, num_images=5):
    max_idx = dataset.shape[0]
    idx = [random.randint(0, max_idx - 1) for _ in range(num_images)]  # random indices

    plt.figure(figsize=(num_images * 3, 3))
    for i, image_idx in enumerate(idx):
        img = dataset[image_idx].reshape(64, 64, 3)  # picks image directly from dataset and reshape to (H, W, C) for display

        plt.subplot(1, num_images, i + 1)
        plt.imshow(img)
        plt.axis("off")


def model_B(dataset, num_images=5):
    # Fit and sample GMM
    gmm = GaussianMixtureWithVarianceFloor(n_components=10, covariance_type = "diag", variance_floor=0.01)
    gmm.fit(dataset)

    samples, _ = gmm.sample(num_images)


    # Plot the sampled images
    plt.figure(figsize=(num_images * 3, 3))
    for i in range(num_images):

        # Reshape to image format
        img = samples[i].reshape(64, 64, 3)
        img = np.clip(img, 0, 1)

        plt.subplot(1, num_images, i + 1)
        plt.imshow(img)
        plt.axis("off")


def model_C(dataset, num_training_img=3, num_images=5):
    max_idx = len(dataset)
    idx = [random.randint(0, max_idx - 1) for _ in range(num_training_img)] # random indicies

    dataset = dataset[idx] # random images from dataset

    # Fit and sample GMM
    gmm = GaussianMixtureWithVarianceFloor(n_components=num_training_img, covariance_type="diag", variance_floor=0)
    gmm.fit(dataset)

    samples, _ = gmm.sample(num_images)

    # Plot the sampled images
    plt.figure(figsize=(num_images * 3, 3))
    for i in range(num_images):

        # Reshape to image format
        samples_img = samples.reshape(num_images, 64, 64, 3)
        samples_img = np.clip(samples_img, 0, 1)

        plt.subplot(1, num_images, i + 1)
        plt.imshow(samples_img[i])
        plt.axis("off")

**Task 1.1.1 (E)** <br/>
Run the `interactive_game` function below. Adjust the `n_samples` parameter to decide how many samples to draw from each model. Start with `n_samples = 1` and gradually increase it and work your way up. Match each row to its generating model. In other words: write which model (A, B, or C) you think corresponds to row 1, 2 and 3.

(Reminder of models: <br/>
Model A: samples directly from the original training dataset.<br/>
Model B: samples from a GMM fitted to the entire training dataset.<br/>
Model C: samples from a GMM fitted to only a few faces and with a low number of GMM-components).<br/>

In [ ]:
interactive_game(model_A, model_B, model_C, birthday_int, dataset_np, n_samples, GaussianMixtureWithVarianceFloor)

When looking at the outputs, we can clearly see that one of the models (one of the rows) produces unnatural and blurry images, while another produces only a very narrow range of outputs – the same few images every time (don’t see it? Experiment more with the number of samples drawn).

These models are good examples of mode collapse and mass-covering behaviour. The blurry and unrealistic images are samples from the model exhibiting **mass-covering**: it spreads its probability mass too widely, generating samples outside the true data distribution, resulting in blurry messes. Meanwhile, the model producing nearly identical images is showing **mode collapse**: it captures only a small subset of the data distribution, failing to represent it fully, resulting in the same few images being sampled.

**Question 1.1.2 (E)** <br/>
How many samples did *you* need to see (what value for `n_samples`) before you could identify which model corresponded to which row? Was it easier to identify the mass covering (too broad) or the mode collapsed (too narrow) model? Explain!

** STUDENT ANSWER HERE **

Now, let us move on to two new models.

This time, we have a completely untrained model, which simply samples from the standard normal distribution, and a model that has memorised a single image.

In [ ]:
def sample_untrained():
    sample = np.random.randn(64, 64, 3)
    return sample

def sample_memorised(mem_img, sigma=0.001):
    """
    Input:
    - mem_img: The memorised image.
    """
    mem_img = mem_img.reshape((64, 64, 3))
    noise = sigma * np.random.randn(*mem_img.shape)
    sample = mem_img + noise
    return sample

Of course, in order to utilise these models, we must must know how to sample from them.

**Task 1.1.3 (E)** <br/>
Sample from the untrained and memorised model. Store the sample from the memorised model in `memo_sample` and from the untrained model in `untr_sample`. Use `memo_img` as the memorised image and `sigma_memo` as the standard deviation.

In [ ]:
# Set random seeds
random.seed(birthday_int)
np.random.seed(birthday_int)

max_idx = dataset_np.shape[0]
memo_img = dataset_np[random.randint(0, max_idx - 1)]   # Random memorised images
sigma_memo = 0.03   # Stdev


In [ ]:
# PUT YOUR CODE HERE FOR TASK 1.1.3

In [ ]:
try:
    memo_sample
    untr_sample
except NameError:
    print("NameError: Did you store you samples in memo_sample and untr_sample?")

### BEGIN HIDDEN TESTS
### END HIDDEN TESTS

Let us also see what exactly it was that we sampled.

In [ ]:
#Reformats data to make it suitable for display
memo_img_disp = memo_img.reshape((64, 64, 3))
memo_img_disp = np.clip(memo_img_disp, 0, 1)
memo_sample_disp = np.clip(memo_sample, 0, 1)
untr_sample_disp = np.clip(untr_sample, 0, 1)


#Dispalying images
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.imshow(memo_img_disp)
plt.title("Original Memorised Image")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(memo_sample_disp)
plt.title("Memorised Sample (with noise)")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(untr_sample_disp)
plt.title("Untrained Sample")
plt.axis("off")

plt.show()

**Question 1.1.4 (E)** <br/>
Which model generates the best-looking faces? Which model is too broad (mass covering) and which model is too narrow (mode collapse)? Motivate your answers.

** STUDENT ANSWER HERE **

While the human eyes are great for many things, they are often not as fault-free as we would often like. Therefore, engineers and scientists often like numerical measures that try to capture some characteristic or property objectively.

One way we could try to quantify how “good” or “bad” a sample is involves using log-likelihood as our objective measure of "goodness". Remember that log-likelihood provides a measure of how probable a given sample is according to the model’s learned distribution. It tells us how well the model explains or assigns probability mass to that particular data point. A higher log-likelihood indicates that the model considers the sample more plausible.

**Task 1.1.5 (E)** <br/>
Evaluate the log-likelihood for both models. For each model, calculate the mean of the log-likelihood over *all* of the images in our training dataset. Store the values of the mean log-likelihood for the memorised model in `ll_memo` and the value for the untrained model in `ll_untr`. You can use the pre-implemented `log_likelihood` function to calculate the log-likelihood.

In [ ]:
def log_likelihood(x, mu, sigma):
    """
    x: np array (H, W, C) - single image
    mu: np array (H, W, C) - mean image
    sigma: float - std deviation
    """
    H, W, C = x.shape
    D = H * W * C  # total dimensionality
    diff = x - mu
    sq_norm = np.sum(diff ** 2)

    ll = - (D / 2) * np.log(2 * np.pi) \
         - (D / 2) * np.log(sigma ** 2) \
         - (1 / (2 * sigma ** 2)) * sq_norm

    return ll

In [ ]:
# PUT YOUR CODE HERE FOR TASK 1.1.5

In [ ]:
try:
    ll_memo
    ll_untr
except NameError:
    print("Did you store the values in ll_memo and ll_untr?")

print(f"Mean log likelihood of memorised model: {ll_memo}")
print(f"Mean log likelihood of untrained  model: {ll_untr}")

### BEGIN HIDDEN TESTS
### END HIDDEN TESTS

**Question 1.1.6 (E)** <br/>
Which model has the highest log-likelihood? The one that is too broad or the one that is too narrow? Compare this with your observations in task and question 1.1.3 - 1.1.4. Was this what you expected? Did the log-likelihood agree with your personal observations using your eyes? Motivate your answer!

** STUDENT ANSWER HERE **

For now, let's return to Model B from Task 1.1.1. Recall that Model B is a GMM fit to the whole training dataset, producing samples with excess diversity and unrealistic, blurry outputs. In deep learning and machine learning in general, temperature is a parameter commonly used during sampling to control randomness and diversity (sometimes viewed as a simple "hack" to control the output).

Now, we will turn to temperature next and explore how temperature affects the samples generated by this GMM model. First, we need to implement sampling with temperature, where the temperature is applied to scale the covariances of the Gaussians.

**Task 1.1.7 (D)** <br/>
Implement GMM sampling with temperature as a parameter. Your function should take **a fitted GMM**, **temperature** and **number of samples to be drawn** as inputs, and output the samples in a numpy array of the form `(n_samples, 64, 64, 3)`.


In [ ]:
def samples_with_temperature(gmm, temperature = 1, n_samples = 4):
    """ Input:
    -gmm: Fitted GMM
    -temperature: the temperature
    -n_samples: number of samples to be drawn.
    Returns: Np array with samples in shape (n_samples, 64, 64, 3)"""

    # PUT YOUR CODE HERE FOR TASK 1.1.7


In [ ]:
try:
    samples_with_temperature
except NameError:
    print("Not implemented")

gmm = GaussianMixtureWithVarianceFloor(n_components=5, covariance_type = "diag", variance_floor=0, random_state = birthday_int)
gmm.fit(dataset_np)

assert samples_with_temperature(gmm, 1, 4) is not None, "Did you return the samples?"
assert np.allclose(samples_with_temperature(gmm, 1, 4), temp_sampling(gmm, 1, 4)), "Error in implementation"

### BEGIN HIDDEN TESTS
### END HIDDEN TESTS

Now, let's draw some samples from our shiny newly implemented model.

**Task 1.1.8 (E)** <br/>
Generate samples with reduced sampling temperature from the already fitted GMM stored in `gmm`. You can use your own function or the pre-implemented `temp_sampling` (if you did not complete task 1.1.7), which takes the same parameters and returns the same output as a correctly implemented `samples_with_temperature`. Sample with temperatures 0.0, 0.25, and 0.5, and store these samples in `samples_0`, `samples_25`, and `samples_50` respectively. Generate **5 samples for each temperature**.


In [ ]:
np.random.seed(birthday_int)
random.seed(birthday_int)

gmm = GaussianMixtureWithVarianceFloor(n_components=10, covariance_type = "diag", variance_floor=0, random_state=birthday_int)
gmm.fit(dataset_np)

n_samples = 5

In [ ]:
# PUT YOUR CODE HERE FOR TASK 1.1.8

In [ ]:
try:
    samples_0
    samples_25
    samples_50
except NameError:
    print("NameError: Have you stored the samples in the correct variables?")

assert samples_0.shape[0] == 5 and samples_25.shape[0] == 5 and samples_50.shape[0] == 5, "Did you draw 5 samples?"

### BEGIN HIDDEN TESTS
### END HIDDEN TESTS

Finally, let's see what the images look like:

In [ ]:
# Draw images
samples_0 = samples_0.reshape(n_samples, 64, 64, 3)
samples_0 = np.clip(samples_0, 0, 1)  # Clip values to valid range if needed

fig1 = plt.figure(figsize=(15, 10))
for i in range(n_samples):
    plt.subplot(4, n_samples, i+1)
    plt.imshow(samples_0[i])
    plt.axis("off")
fig1.suptitle("T=" + str(0), fontsize=16)
plt.tight_layout()
plt.show()

samples_25 = samples_25.reshape(n_samples, 64, 64, 3)
samples_25 = np.clip(samples_25, 0, 1)

fig1 = plt.figure(figsize=(15, 10))
for i in range(n_samples):
    plt.subplot(4, n_samples, i+1)
    plt.imshow(samples_25[i])
    plt.axis("off")
fig1.suptitle("T=" + str(0.25), fontsize=16)
plt.tight_layout()
plt.show()

samples_50 = samples_50.reshape(n_samples, 64, 64, 3)
samples_50 = np.clip(samples_50, 0, 1)

fig1 = plt.figure(figsize=(15, 10))
for i in range(n_samples):
    plt.subplot(4, n_samples, i+1)
    plt.imshow(samples_50[i])
    plt.axis("off")
fig1.suptitle("T=" + str(0.5), fontsize=16)
plt.tight_layout()
plt.show()

**Question 1.1.9 (E)** <br/>
How does the appearance of the faces change when you reduce the sampling temperature? Do you think the faces look better with a temperature close to 1 or close to 0? Motivate your answer!

** STUDENT ANSWER HERE **

## 1.2 Oversmoothing and undersmoothing

Next, we will move on to the concepts of oversmoothing and undersmoothing. When it comes to images, oversmoothing refers to the phenomenon where images become blurry and lose their fine details. Undersmoothing, on the other hand, is the reverse: images appear noisy and may contain too much, often unrealistic, detail. You probably already have an intuitive feeling for what undersmoothed and oversmoothed pictures look like, but to get a feel for this, let's sample some images to look at.

This time, we will again sample from 3 different models.

`data_sample`: A sample directly taken from the dataset.<br/>
`mean_face`: The mean face of the dataset.<br/>
`noisy_sample`: A sample from the data distribution with some added noise.<br/>

(The astute amongst you will remember that `data_sample` is the same as model A from the previous task).

First, let's implement these models:

In [ ]:
def data_sample(dataset, num_images=1):
    """
    Returns:
    - A list containing `num_images` images, each reshaped to (64, 64, 3)
    """
    max_idx = dataset.shape[0]
    idx = [random.randint(0, max_idx - 1) for _ in range(num_images)]  # random indices
    images = []

    for i, image_idx in enumerate(idx):
        img = dataset[image_idx].reshape(64, 64, 3)
        images.append(img)

    return images

def mean_face(dataset, num_images=1, sample_to_average=1000):
    """
    Returns:
    - A list containing `num_images` averaged images, each reshaped to (64, 64, 3)
    """
    max_idx = dataset.shape[0]
    images = []

    for _ in range(num_images):
        idx = np.random.choice(max_idx, sample_to_average, replace=True)
        selected = dataset[idx]

        # Compute mean across selected images
        mean_face = np.mean(selected, axis=0)

        images.append(mean_face.reshape(64, 64, 3))

    return images


def noisy_sample(dataset, num_images=1, sigma = 0.05):
    """
    Returns:
    - A list containing `num_images` noisy images, each reshaped to (64, 64, 3)
    """
    max_idx = dataset.shape[0]
    idx = [random.randint(0, max_idx - 1) for _ in range(num_images)]  # random indices
    images = []

    for i, image_idx in enumerate(idx):
        img = dataset[image_idx].reshape(64, 64, 3)
        noise = sigma * np.random.randn(*img.shape)
        img = img + noise
        images.append(img)

    return images



Now, we can sample from them to see what they produce:

In [ ]:
np.random.seed(birthday_int)
random.seed(birthday_int)

img_q = data_sample(dataset_np, 3)
img_x = mean_face(dataset_np, 3)
img_y = noisy_sample(dataset_np, 3)

# Clip all images before displaying
img_q = [np.clip(img, 0, 1) for img in img_q]
img_x = [np.clip(img, 0, 1) for img in img_x]
img_y = [np.clip(img, 0, 1) for img in img_y]


# Display images
for row in range(3):
    plt.figure(figsize=(9, 9))
    plt.subplot(3, 3, row * 3 + 1)
    plt.imshow(img_q[row])
    plt.title("data_sample")
    plt.axis("off")

    plt.subplot(3, 3, row * 3 + 2)
    plt.imshow(img_x[row])
    plt.title("mean_face")
    plt.axis("off")

    plt.subplot(3, 3, row * 3 + 3)
    plt.imshow(img_y[row])
    plt.title("noisy_sample")
    plt.axis("off")

    plt.tight_layout()
    plt.show()

With the images in hand, let's try to again judge the images with our intuition and eyes.

**Task 1.2.1 (E)** <br/>
Look at each row. For each of the rows, order the images by smoothness from least to most smooth in the code cell below.

In [ ]:
"""
Answer format (example):
most_smooth_r1 = "data_sample"
middle_smooth_r1 = "mean_face"
least_smooth_r1 = "noisy_sample"
"""
#PUT YOUR ANSWERS BELOW FOR TASK 1.2.1

# Row 1:
most_smooth_r1 = ""
middle_smooth_r1 = ""
least_smooth_r1 = ""

# Row 2:
most_smooth_r2 = ""
middle_smooth_r2 = ""
least_smooth_r2 = ""

# Row 3:
most_smooth_r3 = ""
middle_smooth_r3 = ""
least_smooth_r3 = ""

In [ ]:
# Test your answers
answers = [most_smooth_r1, most_smooth_r2, most_smooth_r3, middle_smooth_r1, middle_smooth_r2, middle_smooth_r3, most_smooth_r1, most_smooth_r2, most_smooth_r3]

for answer in answers:
    answer = answer.lower()
    assert answer == "data_sample" or answer == "mean_face" or answer == "noisy_sample", "Have you checked that your answering format is correct?"

While our eyes can perceive whether an image looks smooth or noisy, as we have discussed earlier, it can be very helpful to have a quantitative way to measure smoothness. It is for this purpose, that we will now implement a metric for smoothness. The metric is the standard deviation of pixel intensities in the luminosity (brightness) space of each image. Using luminosity will hopefully help us capture the amount of variation or “texture” in the image: a lower standard deviation in luminosity corresponds to a smoother, more uniform image, while a higher standard deviation indicates more noise or detail.

**Task 1.2.2 (D)** <br/>
Implement a measure of smoothness: the standard deviation in luminosity space. For luminosity, use the following formula:

$L = 0.2126 * R + 0.7152 * G + 0.0722 * B$

(Notice how the weights for red, green and blue is different? That is due to peculiarities in the human eyes, which are much more sensitive to green. This luminosity measure tries to capture that by placing a higher weight on green).

In [ ]:
def luminosity_std(img):
    """
    Input:
    - img: A single image

    Returns
    - Our luminosity measure
    """
    # PUT YOUR CODE HERE FOR TASK 1.2.2

In [ ]:
try:
    luminosity_std
except NameError:
    print("Not Implemented")

# Test case 1
test_img = np.ones((64, 64, 3))
assert luminosity_std(test_img) == 0, "Wrong implementation"

# Test case 2
test_img = np.random.rand(64, 64, 3)
assert luminosity_std(test_img) > 0, "Wrong implementation"

# Test case 3 (half red half blue image)
test_img = np.zeros((64, 64, 3))
test_img[:32, :] = [1, 0, 0]
test_img[32:, :] = [0, 0, 1]
assert np.isclose(luminosity_std(test_img), 0.0702), "Wrong implementation"

### BEGIN HIDDEN TESTS
### END HIDDEN TESTS

Now, that we have our quantifiable measure, let's use our smoothness metric to evaluate the smoothness of the images produced by each model.

**Task 1.2.3 (E)** <br/>
Compute the average smoothness from 1000 samples. The samples are already drawn and stored in `img_a_1000`, `img_b_1000` and `img_c_1000`. You can use your own implementation or the already implemented function `calc_smoothness`(if you have not completed task 1.2.2), which functions exactly like a correctly implemented `luminosity_std`. Store the values in `smth_data_smpl`, `smth_mean_smpl` and `smth_noisy_smpl` respectively.

In [ ]:
np.random.seed(birthday_int)
random.seed(birthday_int)

n_samples = 1000
img_a_1000 = data_sample(dataset_np, n_samples)
img_b_1000 = mean_face(dataset_np, n_samples)
img_c_1000 = noisy_sample(dataset_np, n_samples)

In [ ]:
# Store the smoothness values in these variables.
smth_data_smpl = 0 #data_sample
smth_mean_smpl = 0 #mean_face
smth_noisy_smpl = 0 #noisy sample

# PUT YOUR CODE HERE FOR TASK 1.2.3

print("data_sample smoothness: " + str(smth_data_smpl))
print("mean_face smoothness: " + str(smth_mean_smpl))
print("noisy_sample smoothness: " + str(smth_noisy_smpl))

In [ ]:
assert 1 > smth_mean_smpl > 0 and 1 > smth_mean_smpl > 0 and 1 > smth_noisy_smpl > 0
assert smth_mean_smpl < smth_data_smpl and smth_mean_smpl < smth_noisy_smpl

### BEGIN HIDDEN TESTS
### END HIDDEN TESTS

**Question 1.2.4 (E)** <br/>
Does the standard deviation of luminosity as a measure of smoothness agree with your own opinion of smoothness for the three different models? Explain!

** STUDENT ANSWER HERE **

Next, let's return to some of our models in section 1.1 as well as introduce a new model:  <br/>

`gmm_sampling`: GMM fitted to the entire face dataset. <br/>
`temp_sampling`: gmm_sampling but with reduced temperature. <br/>

Notice how `gmm_sampling` is just our GMM model from task 1.1.1 and `temp_sampling` is just the our model from task 1.1.8 (return to these task if you need a refresher). Below, we have (re-)implemented `gmm_sampling`. For `temp_sampling`, we will call on that function just like we did in task 1.1.8.

In [ ]:
def gmm_sampling(dataset, num_images=5):
    # Fit and sample GMM
    gmm = GaussianMixtureWithVarianceFloor(n_components=10, covariance_type = "diag", variance_floor=0)
    gmm.fit(dataset)

    samples, _ = gmm.sample(num_images)

    return samples

Let's calculate the smoothness for samples from these two additional models. For this, we will set the temperature to 0.25.

In [ ]:
np.random.seed(birthday_int)
random.seed(birthday_int)

temp = 0.25
n_samples = 1000

# Sampling from the models.
img_gmm = gmm_sampling(dataset_np, n_samples)

gmm = GaussianMixtureWithVarianceFloor(n_components=10, covariance_type = "diag", variance_floor=0)
gmm.fit(dataset_np)
img_temp = temp_sampling(gmm, temp, n_samples)

# Reshapes arrays back to (n_samples, 64, 64, 3).
img_gmm = img_gmm.reshape(-1, 64, 64, 3)
img_temp = img_temp.reshape(-1, 64, 64, 3)

# Calculate the average smoothness for each model
smoothness_gmm = 0
smoothness_temp = 0
for i in range(n_samples):
    smoothness_gmm += calc_smoothness(img_gmm[i])
    smoothness_temp += calc_smoothness(img_temp[i])

smoothness_gmm = smoothness_gmm / n_samples
smoothness_temp = smoothness_temp / n_samples

print("gmm_sampling smoothness: " + str(smoothness_gmm))
print("temp_sampling smoothness: " + str(smoothness_temp))

And also: before we continue, let's sample some images so we can get a feel for the models and what they actually output.

In [ ]:
# Clip all images before displaying
img_gmm = [np.clip(img, 0, 1) for img in img_gmm]
img_temp = [np.clip(img, 0, 1) for img in img_temp]


# Display images
for row in range(1):
    plt.figure(figsize=(9, 9))
    plt.subplot(3, 3, row * 3 + 1)
    plt.imshow(img_gmm[row])
    plt.title("gmm_sampling")
    plt.axis("off")

    plt.subplot(3, 3, row * 3 + 2)
    plt.imshow(img_temp[row])
    plt.title("temp_sampling")
    plt.axis("off")

    plt.tight_layout()
    plt.show()

**Question 1.2.5 (E)** <br/>
Are samples from `gmm_sampling` overly smooth? Explain. What about `temp_sampling`?

** STUDENT ANSWER HERE **

**Question 1.2.6 (E)** <br/>
Looking and analysing the results from above, is oversmoothing in this case the result from how the generative model was fit to the data, or how we sampled from it?

** STUDENT ANSWER HERE **

**Question 1.2.7 (E)** <br/>
Looking at the sampled images and the smoothness measure for the two models, does the standard deviation of luminosity as a measure of smoothness agree with your own opinion of smoothness? Explain!

** STUDENT ANSWER HERE **

**Question 1.2.8 (E)** <br/>
Please declare if you used any generative AI tools for solving this notebook, and if so which tools and roughly which parts you used them for. If you did not any generative AI tools, just answer "None". If you did use generative AI tools, also reflect in a sentence or two on how well it worked and what you learnt.

** STUDENT ANSWER HERE **